In [2]:
import random
from duraksub import Card, Hand, Deck, Pair, Discard, Board, BaseGame, BaseAction, BasePlayer

class Player(BasePlayer):
    def __init__(self, game):
        super(BasePlayer, self).__init__()
        self.hand = Hand()
        self.game = game
        
    def __repr__(self):
        att = 'att' if self.game.attacker == self else ''
        df = 'def' if self.game.defender == self else ''
        return f'{self.game.players.index(self)}{att}{df}';
        
class Action(BaseAction):
    def __init__(self, game, player, defender, verb, card=Card(), target=Card()):
        super(BaseAction, self).__init__()
        self.game = game
        self.player = player
        self.defender = defender
        self.verb = verb
        self.card = card
        self.target = target
        
    def compareTo(self, other):
        score = 0
        score += int((self.player == self.defender) == (other.player == other.defender))
        score += int(self.verb == other.verb)
        score += int(self.card == other.card)
        score += int(self.target == other.target)
        return score/4
        
    def randomize(self):
        self.player = random.choice(game.players)
        self.defender = random.choice(game.players)
        self.verb = random.choice(['pass', 'pickup', 'cover', 'play', 'reverse'])
        self.card = Card.random()
        self.target = Card.random()
        
    def __repr__(self):
        return f'Action({self.player}, {self.defender}, {self.verb}, {self.card}, {self.target})';

class Game(BaseGame):
    def __init__(self):
        super(BaseGame, self).__init__()
        self.deck = Deck()
        self.board = Board()
        self.discard = Discard()
        self.players = [Player(self), Player(self), Player(self)]
        self.attacker = self.players[0]
        self.defender = self.players[1]
        self.trump = Card(0,0)
        
    def randomizeBoard(self):
        self.board = Board()
        while random.random() > 0.2:
            a = Card.random()
            b = Card.random() if random.random() > 0.5 else Card()
            self.board.append(Pair(a,b))
            
    def randomizeDiscard(self):
        self.discard = Discard()
        while random.random() > 0.2:
            self.discard.append(Card.random())
            
    def randomize(self):
        self.randomizeBoard()
        self.randomizeDiscard()
        self.trump = Card.random()
        
game = Game()
game.randomizeBoard()
game.randomizeDiscard()

print(game.board)
print(game.discard)

[[15, 33], [3, NoCard], [14, NoCard], [28, 18]]
[14, 18, 22, 29, 22, 32, 27, 9]


In [33]:
from inspect import signature
from itertools import permutations
from functools import partial
from anytree import Node, NodeMixin, RenderTree, PreOrderIter
import duraksub
from duraksub import eq, allowType, MetaSubroutine
import re
import pickle

class Query:
    def __init__(self, verb, params={}, arghints=[]):
        self.verb = verb
        self.params = params # Game, Action
        self.arghints = arghints
        
    def compareTo(self, other):
        score = getNumSimilarWords(self.verb.split(), other.verb.split(), norm=True)
        # Game action-specific queries
        if 'action' in self.params and 'action' in other.params:
            score += self.params['action'].compareTo(other.params['action'])
        return score
    
    def compatible(self, arg):
        return any(allowType(arg, typ) for typ in self.arghints)
    
    @staticmethod
    def fromPrompt(s, game=None):
        def cardParts(s):
            parts = [int(p) for p in s.split(',')]
            if len(parts) != 2:
                raise Exception('Card should be in Rank,Suit format')
            return parts
        parts = s.split()
        params = {}
        if 'game' not in parts:
            game = None
        else:
            params['game'] = game
            del parts[parts.index('game')]
        try:
            idx = parts.index('action')
            verb = ' '.join(parts[:idx])
            parts = parts[idx+1:]
            action = Action(game, game.players[int(parts[0])], game.players[int(parts[1])], parts[2])
            if len(parts) == 4:
                action.card = Card(*cardParts(parts[3]))
            if len(parts) == 5:
                action.target = Card(*cardParts(parts[4]))
            params['action'] = action
        except Exception as ex:
            print(ex)
            verb = ' '.join(parts)
        return Query(verb, params)
            
    def __hash__(self):
        return hash(str(self))
    
    def __eq__(self, other):
        return type(other) == Query and self.verb == other.verb
    
    def __repr__(self):
        game = '' if 'game' in self.params else ' No game'
        return f'Query({self.verb} {self.params.get("action", "No action")}{game})'
    
class NotReadyException(Exception):
    def __init__(self):
        super().__init__()
        
    def __repr__(self):
        return 'Subroutine does not have all args'
            
def makeMetaSubroutine(sub):
    def fn() -> MetaSubroutine:
        return MetaSubroutine(sub)
    fn.__name__ = f'meta{sub.func.__name__.capitalize()}'
    return Subroutine(fn)
    
class Subroutine(NodeMixin):
    def __init__(self, func, query=None, parent=None, children=None, res=None, part=None):
        self.sig = signature(func)
        self.func = func
        self.query = query
        self.parent = parent
        if children is not None:
            self.children = children
        self.res = res
        self.partial = partial(func) if part is None else part
        # For ranking
        self.score = None
    
    def bind(self, params={}):
        bound = {}
        for name in self.sig.parameters:
            if name in params:
                bound[name] = params[name]
        if self.children:
            for child in self.children:
                child.bind(params)
        self.partial = partial(self.func, **bound)
    
    def clean(self):
        for child in self.children:
            child.clean()
        self.res = None
        self.partial = None
        
    def copy(self):
        return Subroutine(self.func, self.query, None, 
                          [child.copy() for child in self.children], self.res, self.partial)
        
    def getUnbound(self):
        return set([name for name in self.sig.parameters]) - set(self.partial.keywords.keys())
        
    def ready(self):
        return ((len(self.partial.keywords) == len(self.sig.parameters)) and 
                all([child.ready() for child in self.children]))
    
    def __call__(self):
        if not self.children:
            if not self.ready():
                raise NotReadyException()
            self.res = self.partial()
        else:
            args = []
            for child in self.children:
                args.append(child())
            self.res = self.partial(*args)
        return self.res
    
    def __eq__(self, other):
        return (type(other) == Subroutine and 
                self.func == other.func and 
                len(self.children) == len(other.children) and
                all(x == y for x,y in zip(self.children, other.children)))
    
    def __repr__(self):
        children = [child.func.__name__ for child in self.children]
        nBound = len(self.partial.keywords) if self.partial is not None else 0
        return f'{str(self.query)} {self.func.__name__} {len(self.sig.parameters)} {nBound} {children} {self.res}'

def getNumSimilarWords(queryBag, targetBag, norm=True):
    score = 0
    for word in queryBag:
        if word in targetBag:
            score += 1
    if norm:
        score /= len(queryBag)
    return score

# https://stackoverflow.com/questions/29916065/how-to-do-camelcase-split-in-python
def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0).lower() for m in matches]

class DurakAI:
    def __init__(self):
        self.subs = [Subroutine(sub) for sub in duraksub.getFunctions(duraksub)]
        self.query = None
        
    def begin(self, query):
        self.accepted = []
        self.rejected = []
        self.failed = []
        self.leaves = []
        self.query = query
        self.rank(query)
        
    def rank(self, query):
        # Compute similarity with query
        for sub in self.subs:
            if sub in self.accepted or sub in self.rejected:
                sub.score = 0
            elif sub.query is not None:
                sub.score = query.compareTo(sub.query)
            else:
                sub.score = getNumSimilarWords(query.verb.split(), camel_case_split(sub.func.__name__))
        # Sort based on similarity
        self.subs.sort(reverse=True, key=lambda a: a.score)
                
    def propose(self):
        idx = 0
        for i,sub in enumerate(self.subs):
            if sub not in self.accepted and sub not in self.rejected:
                idx = i
                break
        sub = self.subs[idx]
        name = str(sub.query) if sub.query is not None else sub.func.__name__
        return (name, sub.score, idx)
    
    def solve(self, game, target):
        if len(self.accepted) == 0:
            raise Exception('No accepted subroutines')
        if self.query is None:
            raise Exception('No query')
        self.query.params['game'] = game
        for sub in self.accepted:
            sub.clean()
            sub.bind(self.query.params)
            if sub.ready():
                sub()
            # TODO allow single-param composite subroutines
            elif len(sub.sig.parameters) == 1:
                self.leaves.append(makeMetaSubroutine(sub.copy()))
                self.leaves[-1]()
            self.leaves.append(sub)
        for i in range(5):
            for sub in self.accepted:
                n = len(sub.getUnbound())
                if n == 0: 
                    continue
                # Trim based on type hints
                hints = [typ for name, typ in sub.func.__annotations__.items() if name != 'return']
                valid = [leaf for leaf in self.leaves 
                         if leaf.res is not None and
                         any(allowType(leaf.res, typ) for typ in hints)]
                perms = list(permutations(valid, n))
                for p in perms:
                    # Check validity of argument types
                    if not all(allowType(readySub.res, typ) for readySub, typ in zip(p, hints)):
                        continue
                    try:
                        csub = sub.copy()
                        csub.children = [readySub.copy() for readySub in p]
                        # Don't recreate tentative sequences
                        if csub in self.leaves:
                            continue
                        res = csub()
                        # None is not a successful return
                        if res is None:
                            continue
                        if eq(res, target):
                            # Check query arghints
                            if len(self.query.arghints) > 0:
                                if not all(self.query.compatible(child.res) for child in csub.children):
                                    continue
                            # Add to accepted and return
                            csub.query = self.query
                            self.accepted.append(csub)
                            return (res, csub, len(self.accepted)-1)
                        # Add to leaves
                        self.leaves.append(csub)
                    except Exception as ex:
                        print(ex)
                        pass
                    except KeyboardInterrupt:
                        print('KeyboardInterrupt')
                        print(f'{n} {len(perms)} {len(leaves)}')
                        raise
                    except:
                        raise
        # Unable to find solution
        return None
    
    def accept(self, idx):
        self.accepted.append(self.subs[idx])
        
    def reject(self, idx):
        self.rejected.append(self.subs[idx])
        
    def suggest(self, keywords):
        q = Query.fromPrompt(' '.join(keywords))
        self.rank(q)
        
    def correct(self, idx):
        self.subs.append(self.accepted[idx])
        
    def incorrect(self, idx):
        self.leaves.append(self.accepted[idx])
        del self.accepted[idx]
        
    def save(self, fname):
        with open(fname, 'wb') as f:
            pickle.dump(self.subs, f)
            
    def load(self, fname):
        with open(fname, 'rb') as f:
            self.subs = pickle.load(f)
        
def printTree(seq):
    for pre, _, node in RenderTree(seq):
        print(f'{pre}{node}')
        
ai = DurakAI()
                        
print('Complete')

Complete


In [7]:
ai.save('ai1.pkl')

AttributeError: Can't pickle local object 'makeConcept.<locals>.fn'

In [43]:
s = input('query: ')
if 'exit' not in s:
    q = Query.fromPrompt(s, game)
    print(q)

    ai.begin(q)

    while True:
        text, score, idx = ai.propose()
        print((text, score, idx))
        s = input('> ')
        if 'exit' in s:
            break
        elif 'accept' in s:
            ai.accept(idx)
        elif 'reject' in s:
            ai.reject(idx)
        elif 'list' in s:
            print(ai.accepted)
            print(ai.rejected)
        elif 'suggest' in s:
            ai.suggest(s.split()[1:])

query: get number of spades on board
'action' is not in list
Query(get number of spades on board No action No game)
('Query(get number of hearts in discard pile No action)', 0.5, 0)
> exit


In [37]:
print(ai.accepted)
print(game.discard)
print(ai.solve(game, 2))

[None getDiscard 1 0 [] None, None count 1 0 [] None, None filter 3 0 [] None, None Hearts 0 0 [] None, None getSuit 1 0 [] None]
[14, 18, 22, 29, 22, 32, 27, 9]
object of type 'Card' has no len()
object of type 'Card' has no len()
object of type 'Card' has no len()


'Suit' object has no attribute 'suit'
(2, Query(get number of hearts in discard pile No action) count 1 0 ['filter'] 2, 5)


In [32]:
for i,sub in enumerate(ai.leaves):
    print(f'{i}. {sub}')

0. None beats 3 0 [] None
1. None getTrump 1 1 [] 0
2. None getTargetA 1 1 [] 7
3. None getCardA 1 1 [] NoCard
4. None beats 3 0 ['getTrump', 'getTargetA', 'getCardA'] False


In [39]:
printTree(ai.accepted[5])

Query(get number of hearts in discard pile No action) count 1 0 ['filter'] 2
└── None filter 3 0 ['getDiscard', 'metaGetsuit', 'Hearts'] [14, 9]
    ├── None getDiscard 1 1 [] [14, 18, 22, 29, 22, 32, 27, 9]
    ├── None metaGetsuit 0 0 [] <duraksub.MetaSubroutine object at 0x000002393A6C9960>
    └── None Hearts 0 0 [] <duraksub.Suit object at 0x000002393A529420>


In [41]:
for i,sub in enumerate(ai.subs):
    print(f'{i}. {sub}')

0. None getSuit 1 0 [] None
1. None getAttacker 1 0 [] None
2. None getBoard 1 0 [] None
3. None getCardA 1 0 [] None
4. None getDefender 1 0 [] None
5. None getDefenderA 1 0 [] None
6. None getHand 1 0 [] None
7. None getIndex 2 0 [] None
8. None getItem 2 0 [] None
9. None getPlayerA 1 0 [] None
10. None getRank 1 0 [] None
11. None getTargetA 1 0 [] None
12. None getTrump 1 0 [] None
13. None getUncovered 1 0 [] None
14. None getVerbA 1 0 [] None
15. None Hearts 0 0 [] <duraksub.Suit object at 0x000002393A529420>
16. None filter 3 0 [] None
17. None count 1 0 [] None
18. None getDiscard 1 1 [] [14, 18, 22, 29, 22, 32, 27, 9]
19. None allSameRank 1 0 [] None
20. None beats 3 0 [] None
21. None contains 2 0 [] None
22. None equal 2 0 [] None
23. None flatten 1 0 [] None
24. None hasRank 2 0 [] None
25. None isPositive 1 0 [] None
26. None isZero 1 0 [] None
27. None lessThan 2 0 [] None
28. None Spades 0 0 [] None
29. None Diamonds 0 0 [] None
30. None Clubs 0 0 [] None
31. None NoCar

In [40]:
ai.correct(5)

In [30]:
duraksub.getTrump(game).suit.value

0

In [31]:
duraksub.getCardA(ai.query.params['action'])

NoCard